In [2]:
import pandas as pd
import numpy as np
import json
import tensorflow as tf
import tensorflow_text as text

tf.get_logger().setLevel('ERROR')

In [5]:
def predict_many(mdl, desc, service_decoder):
    """
    Predicts labels and probabilities for multiple descriptions using a trained model.

    Parameters:
        mdl (object): The trained model used for prediction.
        desc (array-like): The input descriptions to be predicted.
        service_decoder (dictionary): A dictionary of service offerings corresponding to the model's output.

    Returns:
        tuple: A tuple containing arrays of predicted labels and corresponding probabilities.
    """
    probabilities = mdl.predict(x = desc)
    max_prob_idx = np.argmax(probabilities, axis=1)
    probs = np.max(probabilities, axis=1)
    labels = np.array([service_decoder[str(idx)] for idx in max_prob_idx])
    return labels, probs

def predict_one(mdl, desc_str, service_decoder):
    """
    Predicts a label and probability for a single description using a trained model.

    Parameters:
        mdl (object): The trained model used for prediction.
        desc_str (str): The input description to be predicted.
        service_decoder (dictionary): A dictionary of service offerings corresponding to the model's output.

    Returns:
        tuple: A tuple containing the predicted label and the corresponding probability.
    """
    input_desc = [desc_str]
    prob = mdl.predict(x = input_desc)
    idx = np.argmax(prob, axis=1)[0]
    max_prob = np.max(prob, axis=1)[0]
    label = service_decoder[str(idx)]
    return label, max_prob

In [6]:
# Configure the filepath to where you placed these files
model_path = 'saved_model/best_top_25_bert_small'
service_encoder_path = "../src/service_to_idx.json"
service_decoder_path = "../src/idx_to_service.json"

In [7]:
with open(service_encoder_path, "r") as enc:
    service_enc = json.load(enc)

with open(service_decoder_path, "r") as dec:
    service_dec = json.load(dec)

In [8]:
transformer = tf.keras.models.load_model(model_path, compile=False)
transformer.summary()

2023-06-14 16:02:05.720444: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-14 16:02:05.720588: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-14 16:02:05.720691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-14 16:02:05.720789: W tensorflow/compiler/xla/stream_executor/platform/defa

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [9]:
dummy_data = [
    """received from: efgrossm@eng.ucsd.edu Hi UCSD service desk,I am trying to access MyStudentChart, 
    but my phone broke yesterday. I cannot use DuoMobile to confirm login from my computer. 
    Can I get a bypass code while I wait for my new phone to come? -Esther
    """,
    """received from: rsotelo@ucsd.edu

    Hi Service desk,

    Can you please help me register my era commons username RYSOTELO and USCD affiliation to DUO.

    Thank you,

    Raquel
    """,
    "My account is compromised, I need to change my password",
    "Wifi at Sixth college is not working. All of my friends are reporting a problem",
    "My VPN is not working. It is stuck on trying to connect when I type in my password",
    "I am not receiving any emails in my Outlook exchange. The last email I got was 2 months ago. HELP!!!",
    "I just registered for Math 189 yesterday and it is not showing up as a course on my Canvas page. Please help",
    "My docking station is broken, the monitors connected to it does not work",
    "My health email is not receiving any emails, what is going on?",
    "I need Adobe Acrobat for work, how can I get that on my account?"
]
predictions = predict_many(transformer, dummy_data, service_dec)
print(predictions)

1/1 [==============================] - 1s 668ms/step
(array(['MultiFactor Authentication', 'MultiFactor Authentication',
       'Password reset', 'Wireless Residential Networking',
       'Virtual Private Network (VPN)', 'Exchange Online', 'Canvas LMS',
       'Hardware Endpoint Support', 'Health IT Services ',
       'Software Endpoint Support'], dtype='<U31'), array([0.9658683 , 0.38882044, 0.80870456, 0.52833146, 0.83695424,
       0.90501285, 0.9938696 , 0.91025704, 0.39728653, 0.5935595 ],
      dtype=float32))


In [10]:
predict_one(transformer, "My account is compromised, I need to change my password", service_dec)

1/1 [==============================] - 0s 79ms/step


('Password reset', 0.80870456)